# Set up environment and load in data

In [ ]:
library(tidyverse)
library(here)
theme_set(theme_bw())
helpers_path = here()

# YN Task

In [ ]:
data_yn = read.csv(paste0(helpers_path, '/inputs/data_choiceYN.csv'))

Columns according to a readme file:

| Column      | Description |
| ----------- | ----------- |
| subnum | Subject Number |
| day | day number |
| week | week number |
| stimNum  | stimulus number, each stimulus has a identification number |
| stimLettNum | number of the combination of letters in the stimulus, each letter combination has a identification number |
| type  | stimulus' type:   0  == regular exposure; 1  == hypertrained |
| orientation  | orientation angle of the grid |
| filling  | amount of angle filled with color |
| shape | shape number |
| valueO | value in weight of the orientation |
| valueF | value in weight of the filling |
| valueS | value in weight of the shape |
| session | session number |
| yesChosen  | 1 if yes is chosen 0 otherwise |
| response | number of response button for left arrow or right arrow |
| payoff | payoff from trial |
| possiblePayoff | possible payoff from stimulus if yes chosen |
| reference | value of reference amount (if no chosen) can be 0 50 or -50 |
| rt | response time |
| fmri | 1 if the subject was in the scanner during that trial, 0 otherwise |
| startFix | start fixation time when stimulus appears |
| evtimeON | time of the eye tracker on | 
| evtimeOFF | time eye tracker off | 
| crossON | time when the fixation cross is in the middle |

Not in the data that's read in even though it's listed above

| Column      | 
| ----------- | 
| stimLettNum |

Additional columns without descriptions

| Column      | 
| ----------- | 
| inizialTime | 
| endTime | 
| inizialTimeResp | 
| endTimeResp | 

In [ ]:
names(data_yn)

## Visits and number of trials

6 unique subjects

In [ ]:
unique(data_yn$subnum)

Up to 11 sessions per subject

In [ ]:
unique(data_yn$day)

All subjects have 800 trials (4 blocks) of the YN task on each day **except for subject 621 on the 11th day**.

In [ ]:
with(data_yn %>% filter(session != -99), table(subnum, day))

All subjects are **missing a fourth session in the last week** (what should have been visit 12)

In [ ]:
with(data_yn, table(subnum, week))

## Unique stims

There are 726 unique stimuli used (6 shapes x 11 orientations x 11 filling weights)

In [ ]:
length(unique(data_yn$stimNum))

How many unique stimuli does each subject see? Only one subjects sees all stims. Others see almost all of them.

In [ ]:
data_yn %>%
  group_by(subnum) %>%
  summarise(num_unique_stims = length(unique(stimNum)))

## Stim types

On each day each subject encounters 480 HT stimuli and 320 RE stimuli

In [ ]:
with(data_yn, table(subnum, day, type))

Each subject has their unique set of 12 HT stimuli.

In [ ]:
data_yn %>%
  filter(type == 1) %>%
  group_by(subnum) %>%
  summarise(ht_stims = unique(stimNum), .groups="keep")

## Stims by features

Possible values for the orientation attribute:

In [ ]:
sort(unique(data_yn$orientation))

0 and 90 are the most common orientations

In [ ]:
table(data_yn$orientation)

Subjects have differing frequencies of each orientation

In [ ]:
with(data_yn, table(subnum, orientation))

Possible values for the the filling weight:

In [ ]:
sort(unique(data_yn$filling))

**There are 9 unique filling values (not 11)**

In [ ]:
length(unique(data_yn$filling))

0 is the most common filling weight followed by |0.4|

In [ ]:
table(data_yn$filling)

Subjects have differing frequencies of each filling weight. 0 is most common for all but the rest of the order is not the same across subjects.

In [ ]:
with(data_yn, table(subnum, filling))

Number of stims with each orientation and filling combination

In [ ]:
with(data_yn, table(orientation, filling))

Possible values for the stim shape

In [ ]:
unique(data_yn$shape)

Frequency of each shape across all data

In [ ]:
table(data_yn$shape)

Subjects see each shape with different frequencies.

In [ ]:
with(data_yn, table(subnum, shape))

Number of stimuli in each combination of orientation, filling weight and shape

In [ ]:
with(data_yn, table(orientation, filling, shape))

Possible values for the value weight of each orientation

In [ ]:
unique(data_yn$valueO)

Frequency of each orientation value weight

In [ ]:
table(data_yn$valueO)

75 degrees has a value of 0. The further away the orientation from 75 the larger the gain/loss.

In [ ]:
with(data_yn, table(valueO, orientation))

Whether an orientation will have a positive or negative value weight varies for each subject.

In [ ]:
with(data_yn, table(valueO, orientation, subnum))

Possible values for the value weight of each filling weight

In [ ]:
unique(data_yn$valueF)

Frequency of each filling value weight

In [ ]:
table(data_yn$valueF)

Filling weight 0 has 0 value and the further away from 0 has larger gain/loss. All filling weights are associated with one absolute value except for filling weight |0.4| which can have a value of |0.4| or |0.6|.

In [ ]:
with(data_yn, table(valueF, filling))

Whether a filling will have a positive or negative value weight varies for each subject.

In [ ]:
with(data_yn, table(valueF, filling, subnum))

Possible values for the value weight of each shape

In [ ]:
unique(data_yn$valueS)

Frequency of each shape value weight

In [ ]:
table(data_yn$valueS)

Whether a shape will have a positive or negative value weight varies for each subject.

In [ ]:
with(data_yn, table(valueS, shape, subnum))

## Session (Run)

There is an error in the session column.

In [ ]:
unique(data_yn$session)

One session 3 is mis-assigned to -99.

In [ ]:
table(data_yn$session)

**Subject 621 does not have session 3 on day 11.**

In [ ]:
with(data_yn, table(subnum, session))

In [ ]:
data_yn_clean = data_yn %>%
  filter(session != -99)

What exactly are these sessions compared with `day` and `week` columns? They are runs. Subjects complete 4 runs of 200 trials per visit.

In [ ]:
with(data_yn_clean, table(session, week))

In [ ]:
with(data_yn, table(session, day))

## Choice

Choice data is missing for one run so looking at choice only in dataset that excludes run 3 of day 11 for subnum 621.  

In the remaining data the only values for whether "Yes" was chosen are:  

In [ ]:
unique(data_yn_clean$yesChosen)

Response buttons

In [ ]:
unique(data_yn_clean$response)

In [ ]:
with(data_yn_clean, table(fmri,response))

Yes button counterbalanced across subjects.

In [ ]:
with(data_yn_clean, table(yesChosen,response, subnum))

## Payoff

The actual payoffs are higher than the possiblePayoffs presumably because subjects initially reject gambling and learn about the possible payoffs of the stimuli and then choose to play gambles with positive expected value.

In [ ]:
summary(data_yn_clean$payoff)

In [ ]:
summary(data_yn_clean$possiblePayoff)

## Reference value

Frequency of the reference value for when subject chooses "No" for a stim:

In [ ]:
with(data_yn_clean, table(subnum, reference))

In [ ]:
with(data_yn_clean, table(subnum, day, reference))

In [ ]:
with(data_yn_clean, table(subnum, session, reference))

There are the same number of trials for each reference value in the scanner across subjects but these are not equally distributed across subjects. A third of the subjects have seen all reference values in the scanner while the other two thirds have either not seen the 50 or -50 in the scanner.

In [ ]:
with(data_yn_clean, table(subnum, reference, fmri))

## fMRI trials

Number of trials for each run of the task in and out of the scanner across subjects

In [ ]:
with(data_yn_clean, table(subnum, session, fmri))

All subjects have 400 trials (2 runs) per each of the 3 fmri visits

In [ ]:
with(data_yn_clean, table(subnum, day, fmri))

Subjects completed 2nd to 4th sesssions in the scanner, never the first session. **But I'm not sure if subjects who completed sessions 2 and 4 were taken out of the scanner for session 3.**

In [ ]:
with(data_yn_clean %>% filter(fmri==1), table(subnum, session, day))

## RT

Any negative RTs? No.

In [ ]:
sum(data_yn_clean$rt< 0)

Distribution of the RTs.

In [ ]:
summary(data_yn_clean$rt)

There are some trials with very long RTs

In [ ]:
data_yn_clean %>%
  filter(rt>5) %>%
  ggplot(aes(rt))+
  geom_histogram(alpha=0.5, bins=30)+
  theme(panel.grid = element_blank())+
  facet_wrap(~subnum)+
  xlab("")+
  labs(title="RT>5s trials")

Number of trials with longer than 5 s RTs for each subject across visits.

In [ ]:
with(data_yn_clean %>% filter(rt>5), table(subnum, day))

Proportion of all trials with RT > 5s

In [ ]:
nrow(data_yn_clean %>% filter(rt>5))/nrow(data_yn_clean)

What proportion of slow trials is from each subject?

In [ ]:
round(with(data_yn_clean %>% filter(rt>5), table(subnum))/nrow(data_yn_clean %>% filter(rt>5)), 3)

## Blank columns

In [ ]:
for (i in 1:length(names(data_yn))){
  cur_col = names(data_yn)[i]
  if(unique(data_yn[,cur_col])[1] == "-99" && length(unique(data_yn[,cur_col]))==1){
    print(cur_col)
  }
}

Drop the above columns without data

In [ ]:
data_yn_clean = data_yn_clean %>%
  select(-inizialTime, -inizialTimeResp, -endTime, -endTimeResp)

## Eyetracking columns with little description

This is probably when each trial began but I don't know what it is in reference to.

In [ ]:
length(unique(data_yn_clean$startFix))

These are presumably eye-tracking data for fmri trial but no idea what units they are in or what the reference is. I'll keep them but can't use them for any analysis yet.

In [ ]:
length(unique(data_yn_clean$evtimeON))

In [ ]:
length(unique(data_yn_clean$evtimeOFF))

In [ ]:
length(unique(data_yn_clean$crossON))

# BC Task

In [ ]:
data_bc = read.csv(paste0(helpers_path, '/inputs/data_choiceBC.csv'))

Columns according to a readme file:

| Column      | Description |
| ----------- | ----------- |
| subnum | Subject Number |
| day | day number | 
| week | week number | 
| stimNumLeft | stimulus on the left number, each stimulus has a identification number | 
| stimNumRight | stimulus on the right number, each stimulus has a identification number | 
| stimLettNumLeft | number of the combination of letters in the stimulus on the left, each letter combination has a identification number | 
| stimLettNumRight | number of the combination of letters in the stimulus on the right, each letter combination has a identification number | 
| TypeLeft | stimulus' type on the left:   0  == regular exposure; 1  == hypertrained | 
| TypeRight | type of stimulus on the right | 
| OrientationLeft | orientation angle of the grid for stimulus of the left | 
| FillingLeft | amount of angle filled with color for stimulus of the left | 
| ShapeLeft | shape in the stimulus of the left | 
| valueOLeft | value in weight of the orientation for stimulus of the left | 
| valueFLeft | value in weight of the filling for stimulus of the left | 
| valueSLeft | value in weight of the shape on the right |  
| OrientationRight | orientation angle of the grid for stimulus of the right | 
| FillingRight | amount of angle filled with color for stimulus of the right | 
| ShapeRight | shape in the stimulus of the right | 
| valueORight | value in weight of the orientation for stimulus of the right | 
| valueFRight | value in weight of the filling for stimulus of the right | 
| valueSRight | value in weight of the shape on the right |  
| leftChosen | 1 if left option is chosen 0 otherwise | 
| Response | number of response button for left arrow or right arrow | 
| Payoff | payoff from trial | 
| possiblePayoffLeft | possible payoff from stimulus on the left | 
| possiblePayoffRight | possible payoff from stimulus on the right | 
| Rt | response time |  
| fmri | 1 if the subject was in the scanner during that trial, 0 otherwise | 
| startFix | start fixation time when stimulus appears |
| evtimeON | time of the eye tracker on | 
| evtimeOFF | time eye tracker off | 
| crossON | time when the fixation cross is in the middle |

Additional columns without descriptions

| Column      | 
| ----------- | 
| inizialTime | 
| endTime | 
| inizialTimeResp | 
| endTimeResp | 

In [ ]:
names(data_bc)

## Visits and number of trials

6 unique subjects

In [ ]:
unique(data_bc$subnum)

Up to 11 sessions per subject

In [ ]:
sort(unique(data_bc$day))

All subjects have 200 trials of the BC task on each day

In [ ]:
with(data_bc, table(subnum, day))

All subjects are missing a fourth session in the third week (what should have been visit 12)

In [ ]:
with(data_bc, table(subnum, week))

## Unique stims

There are 726 unique stimuli used (6 shapes x 11 orientations x 11 filling weights) on each side

In [ ]:
length(unique(data_bc$stimNumLef))

In [ ]:
length(unique(data_bc$stimNumRight))

How many unique stimuli does each subject see? 

In [ ]:
data_bc %>%
  group_by(subnum) %>%
  summarise(num_unique_left_stims = length(unique(stimNumLef)),
            num_unique_right_stims = length(unique(stimNumRight)))

## Redundant stim num columns

In [ ]:
sum(data_bc$stimLettNumLeft == data_bc$stimNumLef) == nrow(data_bc)

In [ ]:
sum(data_bc$stimLettNumRight == data_bc$stimNumRight) == nrow(data_bc)

Drop redundant columns and fix column name

In [ ]:
data_bc_clean = data_bc %>%
  select(-stimLettNumLeft, -stimLettNumRight) %>%
  rename(stimNumLeft = stimNumLef)

## Stim types

On each day each subject encounters 66 HT stimuli and 134 RE stimuli

In [ ]:
with(data_bc, table(subnum, day, typeLeft))
# with(data_bc, table(subnum, day, typeRight)) #same as above since pairs are of same type

Are pairs of same type (ie. choice is either between *two* HT stimuli or *two* RE stimuli)? Yes.

In [ ]:
with(data_bc, sum(typeLeft == typeRight)) == nrow(data_bc)

## Choice

Possible values for `leftChosen` column. Checking if there are missing trials.

In [ ]:
unique(data_bc$leftChosen)

Possible response buttons in and out of the scanner.

In [ ]:
unique(data_bc$response)

In [ ]:
with(data_bc, table(response, fmri))

Left-right buttons same across subjects

In [ ]:
with(data_bc, table(response, leftChosen, fmri))

## Payoff

Distribution of the earned payoff compared to the possible pay off of each stimuli.

In [ ]:
summary(data_bc$payoff)

In [ ]:
summary(data_bc$possiblePayoffleft)

In [ ]:
summary(data_bc$possiblePayoffright)

## fMRI trials

In [ ]:
with(data_bc, table(subnum, day, fmri))

## RT

Are there negative RTs? No.

In [ ]:
sum(data_bc$rt< 0)

There are again some trials with very long RTs

In [ ]:
summary(data_bc$rt)

There are some trials with very long RTs

In [ ]:
data_bc %>%
  filter(rt>5) %>%
  ggplot(aes(rt))+
  geom_histogram(alpha=0.5, bins=30)+
  theme(panel.grid = element_blank())+
  facet_wrap(~subnum)+
  xlab("")+
  labs(title="RT>5s trials")

Number of trials with longer than 5 s RTs for each subject across visits.

In [ ]:
with(data_bc %>% filter(rt>5), table(subnum, day))

Proportion of all trials with RT > 5s

In [ ]:
nrow(data_bc %>% filter(rt>5))/nrow(data_bc)

What proportion of slow trials is from each subject? Subjects that has the most slow trials in the YN task also are the ones that have the most slow trials in the BC task as well.

In [ ]:
round(with(data_bc%>% filter(rt>5), table(subnum))/nrow(data_bc %>% filter(rt>5)), 3)

## Blank columns

In [ ]:
for (i in 1:length(names(data_bc))){
  cur_col = names(data_bc)[i]
  if(unique(data_bc[,cur_col])[1] == "-99" && length(unique(data_bc[,cur_col]))==1){
    print(cur_col)
  }
}

Drop blank columns

In [ ]:
data_bc_clean = data_bc_clean %>%
  select(-inizialTime, -inizialTimeResp, -endTime, -endTimeResp)

## Eyetracking columns with little description

This is probably when each trial began but I don't know what it is in reference to.

In [ ]:
length(unique(data_bc_clean$startFix))

These are presumably eye-tracking data for fmri trial but no idea what units they are in or what the reference is. I'll keep them but can't use them for any analysis yet.

In [ ]:
length(unique(data_bc_clean$evtimeON))

In [ ]:
length(unique(data_bc_clean$evtimeOFF))

In [ ]:
length(unique(data_bc_clean$crossON))